In [ ]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import struct
from scipy import signal
import pywt
from math import sqrt

%pylab inline

pwd = os.getcwd()
filepath = pwd + "//" + "input_data" + "//" + "test" + "//" + "0" + "//" + "1"
x = 0
data1 = []
data2 = []

interval = 5
dt = 0.0000001 * interval
fs = 10000000 / interval
#start = 250000
#end = 450000
fig_size = 20

with open(filepath, "rb") as fb:
    data = fb.read()

ch1ch2 = struct.unpack("<"+str(int(len(data)/2))+"H", data)
ch1ch2 = np.array(ch1ch2)
ch1ch2 = (ch1ch2-8192)*2.5/8192

datay1 = ch1ch2[::2]
datay2 = ch1ch2[1::2]

start1 = 0
end1 = 0
for index,data in enumerate(datay1):
    if data >= 0.05 and index > 20000:
        start1 = index - 20000
        end1 = index + 180000
        break
        
if start1 == end1 or end1 >= len(datay1):
    start1 = 250000
    end1 = 450000
        
start2 = 0
end2 = 0
for index,data in enumerate(datay2):
    if data >= 0.05 and index > 20000:
        start2 = index - 20000
        end2 = index + 180000
        break
        
if start2 == end2 or end2 >= len(datay2):
    start2 = 250000
    end2 = 450000
        
data1 = datay1[start1:end1:interval]
data2 = datay2[start2:end2:interval]

wavelet = 'morl'
c = pywt.central_frequency(wavelet)
fa = np.arange(400000, 20000, -1000)
scales = np.array(float(c)) * fs / np.array(fa)

[cfs1,frequencies1] = pywt.cwt(data1,scales,wavelet,dt)
[cfs2,frequencies2] = pywt.cwt(data2,scales,wavelet,dt)
power1 = (abs(cfs1)) ** 2
power2 = (abs(cfs2)) ** 2

length_now = len(power1[0])
power1 = np.reshape(power1,(len(power1),fig_size,int(length_now/fig_size)))
power2 = np.reshape(power2,(len(power2),fig_size,int(length_now/fig_size)))
power1 = np.mean(power1,axis=2)
power2 = np.mean(power2,axis=2)

power1 = power1.T
power2 = power2.T

length_now = len(power1[0])
power1 = np.reshape(power1,(fig_size,fig_size,int(length_now/fig_size)))
power2 = np.reshape(power2,(fig_size,fig_size,int(length_now/fig_size)))
power1 = np.mean(power1,axis=2)
power2 = np.mean(power2,axis=2)

power1 = power1.T
power2 = power2.T

power1 = np.log10(power1)
power2 = np.log10(power2)

mx = power1.max()
mn = power1.min()
power1 = (power1-mn) / (mx-mn)
power1 = power1.flatten()


mx = power2.max()
mn = power2.min()
power2 = (power2-mn) / (mx-mn)
power2 = power2.flatten()

In [ ]:
with tf.Session() as sess:  
    
    tf.local_variables_initializer().run()
    tf.global_variables_initializer().run()
    
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    saver = tf.train.import_meta_graph('saver/SAE2-500/SAE.meta')
    saver.restore(sess,'saver/SAE2-500/SAE')  
    h = tf.get_collection('output_y')[0]
    y = tf.get_collection('output_y')[1]
    #l = tf.get_collection('output_y')[2]
    
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("input_x:0")
    
    hidden1,outputdata1 = sess.run([h,y],feed_dict ={x:[power1]})
    hidden2,outputdata2 = sess.run([h,y],feed_dict ={x:[power2]})
    
    power1 = np.reshape(power1,(20,fig_size))
    power2 = np.reshape(power2,(20,fig_size))
    outputdata1 = np.reshape(outputdata1,(20,fig_size))
    outputdata2 = np.reshape(outputdata2,(20,fig_size))
    #hidden1 = np.reshape(hidden1,(8,8))
    #hidden2 = np.reshape(hidden2,(8,8))
    
    
    plt.subplot(2,1,1)
    plt.imshow(power1,cmap=plt.get_cmap('rainbow'))
    plt.axis('off')
    plt.subplot(2,1,2)
    plt.imshow(outputdata1,cmap=plt.get_cmap('rainbow'))
    plt.axis('off')
    plt.figure()
    plt.imshow(hidden1,cmap=plt.get_cmap('rainbow'))
    plt.axis('off')
    plt.show()
    
    plt.subplot(2,1,1)
    plt.imshow(power2,cmap=plt.get_cmap('rainbow'))
    plt.axis('off')
    plt.subplot(2,1,2)
    plt.imshow(outputdata2,cmap=plt.get_cmap('rainbow'))
    plt.axis('off')
    plt.figure()
    plt.imshow(hidden2,cmap=plt.get_cmap('rainbow'))
    plt.axis('off')
    plt.show()
    
    loss1 = sum((power1 - outputdata1)**2)
    loss2 = sum((power2 - outputdata2)**2)
    print(loss1,loss2)
    
#    similar1 = 1 - loss1 * 10
#    similar2 = 1 - loss2 * 10
#    print(similar1,similar2)
    
#     power1 = power1.flatten()
#     power2 = power2.flatten()
#     outputdata1 = outputdata1.flatten()
#     outputdata2 = outputdata2.flatten()
#     power1_norm = sqrt((power1 ** 2).sum())
#     power2_norm = sqrt((power2 ** 2).sum())
#     outputdata1_norm = sqrt((outputdata1 ** 2).sum())
#     outputdata2_norm = sqrt((outputdata2 ** 2).sum())
#     p1o1 = (power1 * outputdata1).sum()
#     p2o2 = (power2 * outputdata2).sum()
#     cos1 = p1o1 / power1_norm / outputdata1_norm
#     cos2 = p2o2 / power2_norm / outputdata2_norm
#     print(cos1,cos2)
    
    #if loss1 > 2:
    #    loss1 = 2
    #if loss2 > 2:
    #    loss2 = 2
    #print("%d%%"%((2-loss1)*50),"%d%%"%((2-loss2)*50))
    
    
    coord.request_stop()
    coord.join(thread)

In [ ]:
print(hidden1)